# Let's create a POC project in parts.

1. Create a docker-compose application that will have all applications
2. Create a OpenWheather API
3. Create a DDL for Clickhouse based on Schema
4. Create a view from pandas transformations
5. Final touch to the project.

### Prompt 1: Create Infrastructure
You are a Data Engineer. Create a docker-compose application that will contain 
clickhouse and graphana. Clickhouse ports should be exposed so that will be accesible 
from outside of container. Respond only with content of `docker-compose.yaml`.

### Prompt 2: Create API Clinet
You are Data Engineer. Create a production ready script that will use OpenWheather API to fetch hourly wheather forecast in 
Poznań, Poland and save it as csv.

In [ ]:
import requests
import os
import pandas as pd
from datetime import datetime

# Constants
API_KEY = os.getenv('API_KEY')
CITY = 'Poznań'
COUNTRY_CODE = 'PL'
BASE_URL = 'http://api.openweathermap.org/data/2.5/forecast'
CSV_FILE_PATH = 'poznan_hourly_weather_forecast.csv'

def get_weather_data(api_key, city, country_code):
    # Construct the API query url
    url = f"{BASE_URL}?q={city},{country_code}&appid={api_key}&units=metric"
    
    # Make a GET request to the OpenWeatherMap API
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching data from OpenWeather API: Status Code {response.status_code}")

def process_weather_data(weather_data):
    # Parse the hourly data
    hourly_data = weather_data['list']
    
    # Create a list to hold our parsed data
    weather_rows = []
    for entry in hourly_data:
        weather_details = {
            'datetime': datetime.fromtimestamp(entry['dt']),
            'temperature': entry['main']['temp'],
            'feels_like': entry['main']['feels_like'],
            'pressure': entry['main']['pressure'],
            'humidity': entry['main']['humidity'],
            'wind_speed': entry['wind']['speed'],
            'description': entry['weather'][0]['description']
        }
        weather_rows.append(weather_details)
        
    # Convert the list into a pandas DataFrame
    return pd.DataFrame(weather_rows)

def save_data_to_csv(data, file_path):
    # Ensure the folder where you want to save the file exists or handle exceptions as needed.
    data.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

def main():
    try:
        # Fetch the weather data from OpenWeather API
        weather_data = get_weather_data(API_KEY, CITY, COUNTRY_CODE)
        
        # Process the weather data into a suitable format
        processed_data = process_weather_data(weather_data)
        
        # Save the processed data to CSV
        save_data_to_csv(processed_data, CSV_FILE_PATH)
        
        print("Hourly weather forecast retrieval complete.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


### Prompt 3:
Create `CREATE TABLE` DDL for Clickhouse from this polars schema:

In [26]:
import pandas as pd
df = pd.read_csv("poznan_hourly_weather_forecast.csv", parse_dates=True)

### Prompt 4:
Write a pandas transformation that will group by day and calculate min, max and avg temp for each day from csv liek this. use pandas

In [30]:
# Read the data into a pandas DataFrame
df = pd.read_csv("poznan_hourly_weather_forecast.csv", parse_dates=True)

# Convert 'datetime' column to datetime objects
df['datetime'] = pd.to_datetime(df['datetime'])

# Set the 'datetime' column as the DataFrame's index
df.set_index('datetime', inplace=True)

# Group the data by day, and calculate min, max, and average temperature
daily_stats = df.groupby(pd.Grouper(freq='D')).agg(
    min_temperature=('temperature', 'min'),
    max_temperature=('temperature', 'max'),
    avg_temperature=('temperature', 'mean')
)

daily_stats

,min_temperature,max_temperature,avg_temperature
datetime,,,
2024-05-07,8.62,15.31,11.802000
2024-05-08,5.07,15.91,10.903750
2024-05-09,6.02,18.57,13.161250
2024-05-10,10.78,16.75,13.336250
2024-05-11,11.22,14.09,12.285000
2024-05-12,5.23,9.59,7.746667


### Prompt 5:
Translate provided pandas transformation to SQL (Clickhouse style) and create a DDL that will create a view.

In [ ]:
CREATE VIEW daily_weather_stats AS
SELECT
    toDate(datetime) as day, -- This casts the datetime to a date, effectively grouping by day
    min(temperature) AS min_temperature,
    max(temperature) AS max_temperature,
    avg(temperature) AS avg_temperature
FROM
    poznan_hourly_weather_forecast
GROUP BY
    day;

# Do Prezentacji!!!!

In [ ]:
from dataclasses import dataclass, asdict
from typing import Union, Dict

def to_dict(input_data: Union[Dict, 'DataclassType']) -> Dict:
    if isinstance(input_data, dict):
        return input_data
    elif hasattr(input_data, '__dataclass_fields__'):  # Checks if input_data is a dataclass
        return asdict(input_data)
    else:
        raise ValueError("Input must be a dictionary or a dataclass instance.")


This is the S3 file URI:
`s3://paida_bucket/presentations/year=2024/month=05/day=05/paida_analytics_20240505190000.csv`

Write a regex pattern that will extract:
collection = "presentations"
partition = "2024-05-05"
table_namne = "paida_analytics"
created_at = "2024-05-05 19:00:00"

In [ ]:
    pattern = r's3://([^/]+)/([^/]+)/year=(\d{4})/month=(\d{2})/day=(\d{2})/([^_]+)_(\d{14})\.csv'

In [ ]:
Generate a sample dataset in csv that will meets following constaints:
- Columns: Id int, first_name str, last_name str, age int, birthday date, created_at datetime
- Names should be Polish
- Age should be between 18 and 50
- All users should have bithday at may 22th.ABORT

Respond only with markdown codeblock and nothing else.
    
Wygeneruj przykładowy zestaw danych w formacie csv, który będzie spełniał następujące warunki:
- Kolumny: id int, first_name str, last_name str, age int, birthday date, created_at datetime
- Imiona i nazwiska powinny być polskie
- Wiek powinien być pomiędzy 18 a 50
- Wszyscy użytkownicy powinni mieć urodziny 22 maja.
    

This is a transformation in pandas. Translate this transformation to polars.


agg_df = df.groupby('product').agg({
    'sales': 'sum',
    'price': 'mean'
}).reset_index()
filtered_df = df[df['price'] > 9.99]
df['rolling_mean_sales'] = df['sales'].rolling(window=3).mean()


In [ ]:
Generate 

In [ ]:
df = df.groupby("product").agg({"sales": "sum", "price": "mean"}).reset_index()
df = df[df["price"] > 9.99]
df["rolling_mean_sales"] = df["sales"].rolling(window=3).mean()

import polars as pl

df = (
    df.groupby("date")
    .agg([pl.sum("sales").alias("sales"), pl.mean("price").alias("price")])
    .filter(pl.col("price") > 9.99)
    .with_column(
        pl.col("sales").rolling_mean(window_size=7).alias("rolling_mean_sales")
    )
)

from dataclasses import dataclass, asdict
from typing import Union, Dict


def to_dict(input_data: Union[Dict, "DataclassType"]) -> Dict:
    if isinstance(input_data, dict):
        return input_data
    elif hasattr(input_data, "__dataclass_fields__"):
        return asdict(input_data)
    else:
        raise ValueError("Input must be a dictionary or a dataclass instance.")

In [9]:
import io
data = io.BytesIO(b"""Id,first_name,last_name,age,birthday,created_at
1,Kacper,Nowak,24,1999-05-22,2023-04-01T14:23:55
2,Aleksandra,Kowalska,31,1992-05-22,2023-04-02T10:45:30
3,Jan,Kaminski,28,1995-05-22,2023-04-01T18:00:12
4,Marta,Wisniewska,35,1988-05-22,2023-04-01T20:15:45
5,Piotr,Wojcik,42,1981-05-22,2023-04-03T09:22:37
6,Julia,Zajac,21,2002-05-22,2023-04-02T12:48:56
7,Tomasz,Kozlowski,38,1985-05-22,2023-04-02T16:33:11
8,Anna,Pawlak,29,1994-05-22,2023-04-01T17:05:42
9,Marcin,Dabrowski,44,1979-05-22,2023-04-02T07:02:00
10,Katarzyna,Lewandowska,19,2004-05-22,2023-04-03T15:40:29""")

import polars as pl

print(dict(pl.read_csv(data).schema))

{'Id': Int64, 'first_name': String, 'last_name': String, 'age': Int64, 'birthday': String, 'created_at': String}


In [10]:
Generate a CREATE TABLE DDL SQL statement for MSSQL Database that will create a table compatible with this polars schema
```{'Id': Int64, 'first_name': String, 'last_name': String, 'age': Int64, 'birthday': String, 'created_at': String}```

SyntaxError: invalid syntax (389514943.py, line 1)

In [ ]:
{'Id': Int64, 'first_name': String, 'last_name': String, 'age': Int64, 'birthday': String, 'created_at': String}

In [ ]:
CREATE TABLE MyTable (
    Id INT NOT NULL,
    first_name NVARCHAR(MAX),
    last_name NVARCHAR(MAX),
    age INT,
    birthday NVARCHAR(MAX),
    created_at NVARCHAR(MAX)
)

In [ ]:
def load_config(environment: str, credentials: Credentials) -> Dict[str, Any]:
    with open(CONFIG_PATH[environment]) as f:
        template = Template(f.read())
        template.globals["secret"] = partial(get_secret, credentials=credentials)
        template.globals["masked_secret"] = get_masked_secret
        render = template.render()
        config = yaml.safe_load(render)

    return config


def load_config(environment: str, credentials: Credentials) -> Dict[str, Any]:
    """
    Load the configuration file based on the specified environment.

    This function reads the configuration file for the given environment and
    generates dynamic variables using Jinja2 templates. Any secrets required 
    in the configuration template are fetched dynamically using the provided 
    credentials.

    Args:
        environment (str): The name of the environment for which the 
            configuration file needs to be loaded.
        credentials (Credentials): The credentials object required to fetch
            any secrets needed in the configuration from a secure storage.

    Returns:
        dict: A dictionary representing the loaded configuration.

    Raises:
        FileNotFoundError: If the configuration file for the provided 
            environment doesn't exist.
        yaml.YAMLError: If there is an error parsing the configuration file.
    """
    with open(CONFIG_PATH[environment]) as f:
        template = Template(f.read())
        template.globals["secret"] = partial(get_secret, credentials=credentials)
        template.globals["masked_secret"] = get_masked_secret
        render = template.render()
        config = yaml.safe_load(render)

    return config